# After A star's death,it could dissapear in the huge space, form a neutron star, make a supernova, or form a black hole. it all dependes on some physical features that a star may have.

<img src="https://www.schoolsobservatory.org/sites/default/files/astro/starcycle.jpg">

# This data has some star feature like:
- Visual Apparent Magnitude of the Star (Vmag)
- Distance Between the Star and the Earth (Plx)
- Standard Error of Plx (e_Plx)
- B-V color index (B-V)
- Spectral type (SpType)
- Absolute Magnitude of the Star (Amag)
# The real target which was published for this data was to spectral classification:

<img src="https://astronomy.com/-/media/Images/Magazine%20Articles/2021/04/ScreenShot20210420at3.19.16PM.jpg?mw=600">

# However, I added some features such as the Tempreature, Luminousity, Coldness, and radius to detect if there are possible Black holes formation in the far future or not.

# here are some info about the features: 
- A hot star has a B-V color index close to 0 or negative, while a cool star has a B-V color index close to 2.0. Other stars are somewhere in between.
- Cold  Star means its age is older

- Dwarf (0) and Giant (1)
- The more luminous an object, the smaller the numerical value of its absolute magnitude

# To Understand the Relation between the feature, I will use a finely distrebuted data to understand the basic features, the I'll use the real dataset.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
import plotly.express as px
import plotly.graph_objects as go
from dataprep.datasets import load_dataset
from dataprep.eda import plot
from dataprep.eda import plot_diff
from dataprep.eda import plot_correlation
from dataprep.eda import plot_missing
import math

# 1) The Example dataset:

In [ ]:
df= pd.read_csv("//Users//omniaelmenshawy//Desktop///Star.csv")

In [ ]:
df.head(5)

In [ ]:
df.describe().T.style.background_gradient(subset=['mean','std','50%','count'], cmap='PuBu')

In [ ]:
df.corr()

## Visualizing eah star by its name:

In [ ]:
df['Type Names']= df['Star type']

In [ ]:
df['Type Names'].replace(0  ,'Brown Dwarf',inplace=True)
df['Type Names'].replace(1  ,'Red Dwarf',inplace=True)
df['Type Names'].replace(2  ,'White Dwarf',inplace=True)
df['Type Names'].replace(3  ,'Main Sequence',inplace=True)
df['Type Names'].replace(4  ,'Supergiant',inplace=True)
df['Type Names'].replace(5  ,'Hypergiant',inplace=True)

## label Encoding for better Visualization:

In [ ]:
df['color']= df['Star color']

In [ ]:
df['Spectral']= df['Spectral Class']

In [ ]:
from sklearn.preprocessing import LabelEncoder
color_le = LabelEncoder()
df['color'] = color_le.fit_transform(df.color)

In [ ]:
color_le.classes_

In [ ]:
Spectral_le = LabelEncoder()
df['Spectral'] = Spectral_le.fit_transform(df.Spectral)

In [ ]:
Spectral_le.classes_

### Now we have:
#### spectral  B = 1 , A = 0, O = 6, M= 5, K=4 , G= 3
#### color Red = 10, Blue = 0, Blue white = 2

In [ ]:
df.columns

## Radius and Star type Relation:
####  From the Below Observation we understand that Radius of each star typr has a huge difference in magnitude, for example, Hyper Giants has the biggest radius, which means that they are the biggest in mass also, while the dwarfs are the smallest stars existed.


In [ ]:
fig = px.scatter(df,
                 x="Spectral", y='Radius(R/Ro)', size="Star type", color="Type Names"
                 , size_max=60,
                 template='plotly_dark', title='Star Type vs Radius')
fig.show()

### Taking the radius as the size to detirmine the real difference in size:

- Now we can observe the real difference.
- For our interest in detecting a black hole, we need the Hypergiant and Supergiant Stars.

In [ ]:
fig = px.scatter(df,
                 x="Star type", y='Radius(R/Ro)', size="Radius(R/Ro)", color="Type Names",
                  size_max=60,
                 template='plotly_dark', title='Star Type vs radius as size')
fig.show()

## Spectral Class and its Tempreature:

- we can clearly notice that classes which starts with M, K, and G are the lowest in Tempreature, and these are the classes which we are interested in as we are looking for old stars, as they are more likly to die soon.

In [ ]:
fig = px.bar(df, x = 'Spectral Class',color="Temperature (K)",hover_name="Type Names", template = 'plotly_dark', title='Spectral Type vs temperature')
fig.show()

## Hertzsprung-Russell Diagram: 
- In the Hertzprung-Russell diagram the temperatures of stars are plotted against their luminosities. The position of a star in the diagram provides information about its present stage and its mass. Stars that burn hydrogen into helium lie on the diagonal branch, the so-called main sequence.
<img src="http://cdn.eso.org/images/screen/eso0728c.jpg">

## Luminosity vs. Surface Temperature in degrees to visualize the Hertzsprung-Russell Diagram:

- Our data has a similar shape as the diagram as the supergiants and hypergiants ar in the top of the graph.
- Other normal starsspends their lives in the main sequence.
- Most of the dwarfsare in the bottom of the graphwith the lowest temperatures and luminosity.

In [ ]:
fig = px.scatter(df, x="Temperature (K)", y="Luminosity(L/Lo)", hover_name="Type Names",
                 color="Type Names",log_x=True, log_y=True, template = 'plotly_dark', title='Luminosity vs. Surface Temperature ')
fig.show()

## Final conclusion from the example data:
- we need the same features in the new data to deteirmine the location of each star in the HR diagram

# The Real dataset:

In [ ]:
path = "//Users//omniaelmenshawy//Desktop///Star39552_balanced.csv"
df = pd.read_csv(path)
df.head(5)

## Converting Columns data type to float values

In [ ]:
df["Vmag"] = pd.to_numeric(df["Vmag"], downcast="float", errors='coerce')
df["Plx"] = pd.to_numeric(df["Plx"], downcast="float", errors='coerce')
df["e_Plx"] = pd.to_numeric(df["e_Plx"], downcast="float", errors='coerce')
df["B-V"] = pd.to_numeric(df["B-V"], downcast="float", errors='coerce')

###  If the sun is the nearist star to us, and its Plx = 8.794 148, so how come there are nearer stars to us?

## Data Report and feature visualization:

In [ ]:
from dataprep.eda import create_report
report = create_report(df, title='My Report')

In [ ]:
report

## Reducing the error:

In [ ]:
threshold = df['e_Plx'].mean() + 0.5

df = df[df['e_Plx'] < threshold ]

## Changing the target to a normal feature as it is not our intereset now:

In [ ]:
df['GiantOrDwarf'] = df['TargetClass']

# Calculating Temperature in Kelvins:

 <img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/e018d8f17c023ec5145774ce2854aa3d9459b94e">

In [ ]:
df["Temp"] = 4600*((1/(0.92*df['B-V']+1.7))+(1/(0.92*df['B-V']+0.62)))

## From kelvins to degrees:
- C = K - 273.15

In [ ]:
#df['Temp']= df['Temp']- 273.15

In [ ]:
df.info()

### Making Sure that our equation is true:

In [ ]:
from __future__ import print_function, division 
from PyAstronomy import pyasl
b = pyasl.BallesterosBV_T()
bv = 0.482
# Convert B-V into effective temperature
teff = b.bv2T(0.482)
print("B-V = {0:4.2f} mag -> Teff = {1:4.0f} K".format(bv, teff))
# Convert effective temperature into B-V color
teff = 4568.0
bv = b.t2bv(teff)
print("Teff = {0:4.0f} K -> B-V = {1:4.2f} mag".format(teff, bv))

In [ ]:
"""
df = df[df.Plx != 0]

#Reindex the dataframe
df = df.reset_index(drop=True)

df
"""

## Calculating the Luminosity as L/Lo:

- Absolute magnitude and luminosity are related with the formula

### M = -2.5 * log₁₀(L / L₀)

### where:

- M is the absolute magnitude of the star;
- L is its luminosity;
- L₀ is the zero-point luminosity, equal to 3.0128 * 10²⁸ W.

In [ ]:
df["Luminosity"] = ((10)**(df['Amag']/(-2.5)))
                    #/((3.0128)*(10)**28))

In [ ]:
df

## Leaving only L values > 0 for further calculations:

In [ ]:
df = df[df.Luminosity > 0]

#Reindex the dataframe
df = df.reset_index(drop=True)

# Claculating the radius of the star in meters: 

<img src="https://i.imgur.com/tuKNRDE.png">


In [ ]:
df['Radius']= np.sqrt(((df['Luminosity'])*(3.827*(10**26)))/(4*3.141593*(5.7*(10**(-8)))*(df['Temp']**4)))

In [ ]:
df.columns

a star with a mass greater than 20 times the mass of our Sun may ultimately become a black hole;
V(orbit) = SQRT(GM/R) 

In [ ]:
df.describe().T

# Is the Star Giant or Dwarf?

In [ ]:
df['GiantOrDwarf'].value_counts()

In [ ]:
colors_blue = ["#132C33", "#264D58", '#17869E', '#51C4D3', '#B4DBE9']
colors_dark = ["#1F1F1F", "#313131", '#636363', '#AEAEAE', '#DADADA']

In [ ]:
d= pd.DataFrame(df['GiantOrDwarf'].value_counts())
fig = px.pie(d,values='GiantOrDwarf',names=['Giants','Dwarfs'],hole=0.4,
            color_discrete_sequence=[colors_blue[3],colors_dark[3]],
             labels={'label':'GiantOrDwarf','GiantOrDwarf':'No. Of Samples'})


fig.add_annotation(text='Need to resample the data<br> to get a balanced dataset',
                   x=1.2,y=0.9,showarrow=False,font_size=12,opacity=0.7,font_family='arial')
fig.add_annotation(text='Classified Stars',
                   x=0.5,y=0.5,showarrow=False,font_size=12,opacity=0.7,font_family='arial')

fig.update_layout(
    font_family='arial',
    title=dict(text='Star Types',x=0.47,y=0.98,
               font=dict(color=colors_dark[2],size=20)),
    legend=dict(x=0.37,y=-0.05,orientation='h',traceorder='reversed'),
    hoverlabel=dict(bgcolor='white'))

fig.update_traces(textposition='outside', textinfo='percent+label')

# We are inetersted in the Hot Giants
## Cold Spectral Types are: M,K, and G.

In [ ]:
df['ColdTypes'] = df['SpType']

In [ ]:
def ColdTypes(row):
    if row['ColdTypes'].startswith('M') or row['ColdTypes'].startswith('K') or row['ColdTypes'].startswith('G'):
        return 1
    else:
        return 0

In [ ]:
df['ColdTypes'] = df.apply (lambda row: ColdTypes(row), axis=1)

In [ ]:
df

In [ ]:
d= pd.DataFrame(df['ColdTypes'].value_counts())
fig = px.pie(d,values='ColdTypes',names=['Cold','Hot'],hole=0.4,
            color_discrete_sequence=[colors_blue[3],colors_dark[3]],
             labels={'label':'GiantOrDwarf','GiantOrDwarf':'No. Of Samples'})


fig.add_annotation(text='Need to resample the data<br> to get a balanced dataset',
                   x=1.2,y=0.9,showarrow=False,font_size=12,opacity=0.7,font_family='arial')
fig.add_annotation(text='Classified Stars',
                   x=0.5,y=0.5,showarrow=False,font_size=12,opacity=0.7,font_family='arial')

fig.update_layout(
    font_family='arial',
    title=dict(text='Star Types',x=0.47,y=0.98,
               font=dict(color=colors_dark[2],size=20)),
    legend=dict(x=0.37,y=-0.05,orientation='h',traceorder='reversed'),
    hoverlabel=dict(bgcolor='white'))

fig.update_traces(textposition='outside', textinfo='percent+label')

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
print(df["Amag"].plot.kde())

# Trying to understand Some feature Relations

In [ ]:
fig = px.scatter(df, x="Temp", y="Luminosity",
                 color="GiantOrDwarf",log_x=True, log_y=True, template = 'plotly_dark', title= 'Temperatue vs Luminosity')
fig.show()

In [ ]:
fig = px.scatter(df, x="Temp", y="Amag",
                 color="B-V",log_x=True, log_y=True, template = 'plotly_dark', title= ' Amag Vs. Temp')
fig.show()

## Stellar (PlX): parallax is the apparent shift of position of any nearby star (or other object) against the background of distant objects.

## let us calculate the distance between each blackhole and earth

<img src="http://astronomyonline.org/Science/Images/Mathematics/Parallax.gif">
<img src="https://excelnotes.com/wp-content/uploads/2020/09/parsec-to-kilometer.jpg">


In [ ]:
df['Distance(Km)'] = ((1/df['Plx'])*(3.08567758149137*(10**13)))

In [ ]:
df['Distance(parsec)'] = ((1/df['Plx']))

In [ ]:
df.describe().T

# We are interested in the Mass as we need the Biggest Star Masses only to be counted as the stars which could form a black hole according to ( Schwardzchild radius law) must be 30 times bigger than the sun:

# I need my stars to be in the main sequence to be able to calculate their Mass correctly as: 
<img src="http://www.astronomy.ohio-state.edu/~pogge/Ast162/Unit2/Images/MLrel_eqn.gif">

- Main sequence stars have a Morgan-Keenan luminosity class labelled V

In [ ]:
df

In [ ]:
def Main_Seq(row):
    if row['GiantOrDwarf'] == 1 :
        for i in range(len(df['SpType'])):
            if "V" in df.loc[i,'SpType']:
                return 1
               
    else:
        return 0
    

In [ ]:
df['MainSeq'] = df.apply (lambda row: Main_Seq(row), axis=1)

In [ ]:
df["MainSeq"] = df["MainSeq"].fillna(0)

In [ ]:
df['MainSeq'].value_counts()

- Sun Lum = 3.846 × 10**26
- Sun Mass = 1.9891e30

In [ ]:
df['Mass'] = (np.sqrt(np.sqrt(df['Luminosity']/(3.846*(10**26)))))*(1.9891*(10**30))

In [ ]:
fig = px.scatter(df,
                 x="B-V", y='Mass', size="Mass",color='GiantOrDwarf',
                  size_max=60,
                 template='plotly_dark', title='Star Type vs Mass as size')
fig.show() 

In [ ]:
df.describe()

## Finding the sun in the dataset:
 
- we know that Sun's SpType is G2V and Its BV is 0.65

In [ ]:
df.loc[df['B-V'] == 0.650] #& bool_series = df["SpType"].str.startswith(search,na = False)

In [ ]:
df.loc[df['Plx'] == 742.119995]

### Nearist estimation is column 33043 with mass = 5.225973e+21

In [ ]:
""""
# String to be searched in start of string 
search ="G2V"
  
# boolean series returned
bool_series = df["SpType"].str.startswith(search,na = False)
df[bool_series].head(50)
"""

## Now let us Calculate Our Estimated Star's Mass which could form a black hole:

In [ ]:
x = 5.225973*(10**21)
b= (x)*30
b 

In [ ]:
df.describe()

# Now we have all the values that could probably form a black hole as:

- if the star is a Hot type and giant (Like in the Hertzsprung–Russell diagram) [in our data hot = 0].
- if the stars B-V below  1 which means its color is reaching the blue
- If it is a giant star..
- if its a main sequence star.
- if it mass is 30 times bigger than our estimated sun in this data.

<img src="https://news.harvard.edu/wp-content/uploads/2020/03/F1.large-2-1200x800.jpg">

# Now, let us create the Possible_Black_Holes Function:

In [ ]:
def Possible_Black_Holes(row):
    if row['GiantOrDwarf'] == 1 :
        if row['Mass'] >= b :
            return 1
               
    else:
        return 0

In [ ]:
df['BlackHoles'] = df.apply (lambda row: Possible_Black_Holes(row), axis=1)

In [ ]:
df["BlackHoles"] = df["BlackHoles"].fillna(0)

In [ ]:
df['BlackHoles'].value_counts()

In [ ]:
y = 7/36181
print ('the percentage of a Black hole formation in this data is: ' ,y * 100, '%')

## Let us Visualize Our Possible Future Black Holes:

In [ ]:
Positive = df[df.BlackHoles == 1]

In [ ]:
Positive

In [ ]:
plot(df, 'BlackHoles')

In [ ]:
plot(Positive)

In [ ]:
d= pd.DataFrame(df['BlackHoles'].value_counts())
fig = px.pie(d,values='BlackHoles',names=['Normal','Black Holes'],hole=0.4,
            color_discrete_sequence=[colors_blue[3],colors_dark[1]],
             labels={'label':'BlackHoles','BlackHoles':'No. Of Samples'})


fig.add_annotation(text='Need to resample the data<br> to get a balanced dataset',
                   x=1.2,y=0.9,showarrow=False,font_size=12,opacity=0.7,font_family='arial')
fig.add_annotation(text='Possible Black Holes',
                   x=0.5,y=0.5,showarrow=False,font_size=12,opacity=0.7,font_family='arial')

fig.update_layout(
    font_family='arial',
    title=dict(text='Star Types',x=0.47,y=0.98,
               font=dict(color=colors_dark[2],size=20)),
    legend=dict(x=0.37,y=-0.05,orientation='h',traceorder='reversed'),
    hoverlabel=dict(bgcolor='white'))

fig.update_traces(textposition='outside', textinfo='percent+label')

In [ ]:
fig = px.scatter(Positive,
                 x="Temp", y='Luminosity', size="BlackHoles", color="SpType",
                 log_x=True, size_max=60,
                 template='plotly_dark', title='Possible 7 Black Holes Types, Luminosity, and Amag')
fig.show()

In [ ]:

df.columns

In [ ]:
fig = px.scatter(Positive, y="Plx", x="BlackHoles",
                 color="BlackHoles",log_x=True, log_y=True, template = 'plotly_dark', title='Plx of the Possible Black holes')
fig.show()

In [ ]:
fig = px.scatter_3d(Positive, x="Plx", y='BlackHoles', z='Luminosity',
              color='B-V', template = 'plotly_dark', title = 'Possible Black hole s Plx and Luminosity')
fig.show()

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(20,6))

sns.kdeplot(data = df['Amag'].loc[df['BlackHoles']==0], shade=True, ax = axes[0],label='Negative')
sns.kdeplot(data = df['Amag'].loc[df['BlackHoles']==1], shade=True, ax = axes[0],label='Positive')
axes[0].legend()

axes[0].set_xlabel('Absolute Magnitude', size=14)
sns.kdeplot(data = df['Vmag'].loc[df['BlackHoles']==0], shade=True, ax = axes[1],label='Negative')
sns.kdeplot(data = df['Vmag'].loc[df['BlackHoles']==1], shade=True, ax = axes[1],label='Positive')
axes[1].legend()

axes[1].set_xlabel('Apparent Magnitude', size=14)

## Let Us Try Modelling!

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report

In [ ]:
# Separate the labels
Positive = df[df.BlackHoles == 1]
Negative = df[df.BlackHoles == 0]

In [ ]:
num_of_positive = Positive.shape[0]
num_of_negative = Negative.shape[0]
print("Positive(1):",num_of_positive)
print("Negative(0):",num_of_negative)

## Balancing the Data:

In [ ]:
from sklearn.utils import resample

In [ ]:
# Downsample majority class
df_positive_downsampled = resample(Positive, 
                                 replace=True,    # sample without replacement
                                 n_samples=num_of_negative,     # to match minority class
                                 random_state=1) # reproducible results
 
# Combine minority class with downsampled majority class
df_downsampled = pd.concat([df_positive_downsampled, Negative])

In [ ]:
df_downsampled['BlackHoles'].value_counts()

In [ ]:
sns.countplot(df_downsampled['BlackHoles'])

In [ ]:
df.columns

In [ ]:
y = df['BlackHoles']

# Select Features
x = df[['B-V','Amag','GiantOrDwarf','ColdTypes','Radius', 'Plx', 'MainSeq', 'Mass' ]]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.35, random_state=0)

In [ ]:
scaler = StandardScaler()

In [ ]:
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [ ]:
x_train.shape

In [ ]:
x_test.shape

- FN = there is a black hole and we said that it is a normal star (recall)
- FP = There is a normal star and we said that it is a black hole (precision)
- we need the lowest FN 
- so we need the highst Recall
<img src="https://cdn-images-1.medium.com/max/950/1*PPgItHcPSaskyjLMWFC-Kw.png">


## LogisticRegression

In [ ]:
star_predictor = LogisticRegression(random_state=0)

In [ ]:
star_predictor.fit(x_train, y_train)

In [ ]:
print('the score on train dataset is') 
print(star_predictor.score(x_train, y_train))

In [ ]:
y_pred = star_predictor.predict(x_test)

In [ ]:
print("Test Accuracy : ", accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred),annot=True,fmt='',cmap='YlGnBu')

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_test, y_pred)


In [ ]:
plt.plot(fpr, tpr)  
plt.xlabel("False Positive Rate")  
plt.ylabel("True Positive Rate")  
plt.plot([0, 1], [0, 1], "k--")

plt.show()


## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
star_predictor = KNeighborsClassifier(n_neighbors=3)
star_predictor.fit(x_train, y_train)

In [ ]:
print('the score on train dataset is') 
print(star_predictor.score(x_train, y_train))

In [ ]:
y_pred = star_predictor.predict(x_test)

In [ ]:
print("Accuracy : ", accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred),annot=True,fmt='',cmap='YlGnBu')

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
plt.plot(fpr, tpr)  
plt.xlabel("False Positive Rate")  
plt.ylabel("True Positive Rate")  
plt.plot([0, 1], [0, 1], "k--")

plt.show()


## GaussianNB

In [ ]:
star_predictor = GaussianNB()
star_predictor.fit(x_train, y_train)

In [ ]:
print('the score on train dataset is') 
print(star_predictor.score(x_train, y_train))

In [ ]:
y_pred = star_predictor.predict(x_test)

In [ ]:
print("Accuracy : ", accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred),annot=True,fmt='',cmap='YlGnBu')

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
plt.plot(fpr, tpr)  
plt.xlabel("False Positive Rate")  
plt.ylabel("True Positive Rate")  
plt.plot([0, 1], [0, 1], "k--")

plt.show()



In [ ]:
#from sklearn.inspection import permutation_importance

#star_predictor.feature_importances_

## SVC

In [ ]:
from sklearn.svm import SVC
star_predictor = SVC()
star_predictor.fit(x_train, y_train)  

In [ ]:
print('the score on train dataset is') 
print(star_predictor.score(x_train, y_train))

In [ ]:
y_pred = star_predictor.predict(x_test)

In [ ]:
print("Accuracy : ", accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred),annot=True,fmt='',cmap='YlGnBu')

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
plt.plot(fpr, tpr)  
plt.xlabel("False Positive Rate")  
plt.ylabel("True Positive Rate")  
plt.plot([0, 1], [0, 1], "k--")

plt.show()



## ANN

In [ ]:
import tensorflow as tf

In [ ]:
ann = tf.keras.models.Sequential()

In [ ]:
ann.add(tf.keras.layers.Dense(units = 64, activation = 'relu'))
ann.add(tf.keras.layers.Dense(units = 32, activation = 'relu'))
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

In [ ]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics= ['accuracy'])

In [ ]:
ann.fit(x_train, y_train, epochs=100, batch_size=32)

In [ ]:
y_pred = ann.predict(x_test)
y_pred = (y_pred > 0.5)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred),annot=True,fmt='',cmap='YlGnBu')

In [ ]:
print("Accuracy : ", accuracy_score(y_test, y_pred))

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
plt.plot(fpr, tpr)  
plt.xlabel("False Positive Rate")  
plt.ylabel("True Positive Rate")  
plt.plot([0, 1], [0, 1], "k--")

plt.show()

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
star_predictor = RandomForestClassifier(max_depth=4, random_state=0)
star_predictor.fit(x_train, y_train)  

In [ ]:

print('the score on train dataset is') 
print(star_predictor.score(x_train, y_train))

In [ ]:
y_pred = star_predictor.predict(x_test)

In [ ]:
print("Accuracy : ", accuracy_score(y_test, y_pred))

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
sns.heatmap(confusion_matrix(y_test, y_pred),annot=True,fmt='',cmap='YlGnBu')

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
plt.plot(fpr, tpr)  
plt.xlabel("False Positive Rate")  
plt.ylabel("True Positive Rate")  
plt.plot([0, 1], [0, 1], "k--")

plt.show()



In [ ]:
df.shape

In [ ]:
df['MainSeq'].value_counts()

In [ ]:
17286 / 7

# KNN is the best algorithm that fits our algorithm perfctly with 0 Miss-classified  Possible Black holes and 1 wrongly classified star.


## "Roughly one out of every thousand stars that form is massive enough to become a black hole, Most of these are invisible to us, and only about a dozen have been identified".

- https://www.stsci.edu/~marel/black_holes/encyc_mod3_q7.html 


## In our data, out of 36188 record, and  17286 in the main sequence which we done our estimation on, there are 11 possible black hole. which means that roughly in every 2469 main sequence star star there is a possible black hole detection.

# Scientific resources:


- Coursera Astronomy Data Driven Course From The University of Sydeny: 
#### https://www.coursera.org/learn/data-driven-astronomy/home/welcome


- Hertzsprung-Russell Diagram:
#### https://astronomy.swin.edu.au/cosmos/h/hertzsprung-russell+diagram


- Star Mass:
#### https://www.researchgate.net/figure/Mass-radius-relation-for-Kepler-red-giants-with-RGB-stars-in-blue-and-clump-stars-in_fig6_257882348


- Black Hole propertries: 
#### https://public.nrao.edu/ask/properties-of-stars-which-result-in-black-holes/
#### https://public.nrao.edu/ask/what-is-the-critical-mass-at-which-a-star-becomes-a-black-hole/


- PRINCIPLES OF STELLAR STRUCTURE (OHIO State University)
#### http://www.astronomy.ohio-state.edu/~dhw/Intro/lec6.html
#### http://www.astronomy.ohio-state.edu/~pogge/Ast162/Unit2/structure.html


- Finding Our Sun From Amag: 
#### https://arxiv.org/abs/1804.07788 


- Blackbody Radiation and Quantization of Energy fot Luminosity and B-V:
#### https://web.njit.edu/~gary/321/Lecture2.html


- Finding Temperature and Spectral Type Based on Ballesteros equation: 
#### http://astro.physics.uiowa.edu/ITU/labs/professional-labs/photometry-of-a-globular/part-2-finding-temperature.html


- Calculating the Radius of a Star:
#### https://cas.sdss.org/dr4/en/proj/advanced/hr/radius1.asp


- Brightest Stars: Luminosity & Magnitude:
#### https://www.space.com/21640-star-luminosity-and-magnitude.html


- Distance Parallax equation:
#### http://astronomyonline.org/Science/Parallax.asp 


